In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(15) #set seed for reproducibility purposes
x = np.arange(10) 
y = 2*x + 5 + np.random.randn(10) #generate some data with random gaussian scatter
dy = np.random.rand(10) #these are the uncertainties

In [ ]:
plt.scatter(x,y,marker='.',c='k');

In [ ]:
plt.errorbar(x,y,yerr=dy,marker='.',c='k',ls='None');

In [ ]:
# This fits the data without taking uncertainties into account
from scipy import stats
slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

In [ ]:
slope, intercept

In [ ]:
plt.scatter(x,y,marker='.',c='k')
plt.plot(x, slope*x+intercept,'r-',label = 'Fit with no errors');

In [ ]:
slope2,intercept2 = np.polyfit(x,y,deg=1,w=1/dy)
slope2, intercept2

In [ ]:
plt.errorbar(x,y,yerr=dy,marker='.',c='k',ls='None');
plt.plot(x, slope*x+intercept,'r-',label = 'Fit with no errors')
plt.plot(x, slope2*x+intercept2,'g-',label = 'Fit with errors')
plt.legend();


### Model fitting from scratch. 

Let's now implement the chi2 approach.

The parameters I want to fit are slope, intercept.

Just by looking at the data I can tell that the slope should be somewhere between 1 and 3, and the intercept between 2.5 and 6.5.

Our grid of possible models would then be

In [ ]:
slopes = np.linspace(1,3,100) 
intercepts = np.linspace(2.5,6.5,100)
#note: these are already 10,000 models (curse of dimensionality!)

For convenience, we can define two functions that describe our model (a straight line) and the chi2 function:

In [ ]:
def model(x,m,b):
    return m*x+b #straight line

def chi2(m,b,x,y,err):
    return np.sum(((model(x,m,b) - y)**2)/err**2)

This line calculate chi2 values for our 10,000 combinations

In [ ]:
allchi2 = np.array([[chi2(m,b,x,y,dy) for m in slopes] for b in intercepts])

In [ ]:
allchi2.shape

The next step is to figure out what (slope,intercept) pair gives the minimum chi2. This can be done with np.argmin 
but the index of the minimum is calculated after flattening the array, so we need to use a trick to get the indices for rows and columns.

In [ ]:
print(allchi2.min()) #min chi2 value
print(allchi2.argmin()) #index of min on flattened array
print(np.unravel_index(allchi2.argmin(), allchi2.shape)) #indices of minimum value as a (row, col) pair

In [ ]:
#note that the inner (second) index is for slopes, the outer(first) index is for intercepts

print(slopes[57],intercepts[18]) #comes up pretty close to the result of np.polyfit.